In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import os

def read_data():
    '''
    loop through all CSVs in data folder and read them into a pandas dataframe
    '''
    data = {}
    for file in os.listdir("data"):
        if file.endswith(".csv"):
            df = pd.read_csv("data/" + file)
            curncy_name = file.split(" ")[0].split("_")[-1]
            df.set_index(df["Date"], inplace=True)
            df.index = pd.to_datetime(df.index)
            df.drop(["Date"], axis=1, inplace=True)
            data[curncy_name] = df
    return data
    
    
data = read_data()


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
!po

In [ ]:
def eigen_value_graph(data):
    eigvals = np.linalg.eigvals(data.cov())
    plt.plot(range(len(eigvals)), eigvals)
    plt.xlabel("Eigenvalue Index")
    plt.ylabel("Eigenvalue Magnitude")
    plt.title("Eigenvalues of Predictors")
    plt.show()
    plt.savefig("Eigenvalues")


def get_reduced_data(x):
    pca = PCA(n_components=7)
    data_transformed = pca.fit_transform(x)
    return pca, data_transformed

eigen_value_graph(data)        

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, accuracy_score

def train_regression_models(x_train, x_test, y_train, y_test):
    # Initialize the regression models
    linear_model = LinearRegression()
    decision_tree_model = DecisionTreeRegressor()
    random_forest_model = RandomForestRegressor()
    svm_model = SVR()

    # Train the models using the training data
    linear_model.fit(x_train, y_train)
    decision_tree_model.fit(x_train, y_train)
    random_forest_model.fit(x_train, y_train)
    svm_model.fit(x_train, y_train)

    # Evaluate the models using the test data
    linear_model_pred = linear_model.predict(x_test)
    decision_tree_model_pred = decision_tree_model.predict(x_test)
    random_forest_model_pred = random_forest_model.predict(x_test)
    svm_model_pred = svm_model.predict(x_test)

    # Compute the mean squared error of each model
    linear_model_mse = mean_squared_error(y_test, linear_model_pred)
    decision_tree_model_mse = mean_squared_error(y_test, decision_tree_model_pred)
    random_forest_model_mse = mean_squared_error(y_test, random_forest_model_pred)
    svm_model_mse = mean_squared_error(y_test, svm_model_pred)


    lstm_units = 50
    batch_size = 20
    epochs = 15
    learning_rate = 0.001
    x_train_lstm = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test_lstm = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    model = Sequential()
    model.add(LSTM(units=lstm_units, input_shape=(x_train.shape[1], 1), return_sequences=False))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    # Train the model
    model.fit(x_train_lstm, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

    # Make predictions
    y_pred = model.predict(x_test_lstm)

    # Calculate mean squared error
    lstm_model_mse = mean_squared_error(y_test, y_pred)

    # Create a dataframe to store the results
    results_df = pd.DataFrame({
        'Model': ['Linear Regression', 'Decision Tree', 'Random Forest', 'SVM'],
        'Mean Squared Error': [linear_model_mse, decision_tree_model_mse, random_forest_model_mse, svm_model_mse],
        'Model Object': [linear_model, decision_tree_model, random_forest_model, svm_model]
    })
    results_df.sort_values(by='Mean Squared Error', inplace=True)

    return results_df

In [ ]:
def split_data_by_time(data, label, train_size, pca=False):
    # Split the data into features (x) and target (y) based on the specified label
    x = data.drop(label, axis=1)[:-1]
    y = data[label]
    y = y.shift(-1)[:-1]
    if pca:
        _, x = get_reduced_data(x)
    
    train_index = int(len(data) * train_size)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1-train_size, shuffle=False)

    return x_train, x_test, y_train, y_test



In [ ]:
for curncy in list(data.keys()):
    x_train, x_test, y_train, y_test = split_data_by_time(data, f"{curncy} Curncy", 0.7, pca=True)
    print(train_regression_models(x_train, x_test, y_train, y_test))


In [ ]:
results = {}
strat_errors = {}
for curncy in list(data.keys()):
    x_train, x_test, y_train, y_test = split_data_by_time(data[curncy], f"{curncy} Curncy", 0.7, True)
    models = train_regression_models(x_train, x_test, y_train, y_test)
    model = models.iloc[1]["Model Object"]
    predicted_returns = model.predict(x_test)
    lagged_returns = pd.Series(predicted_returns)
    training_predictions = model.predict(x_train)
    training_score = accuracy_score([1 if y > 0 else 0 for y in y_train], [1 if y > 0 else 0 for y in training_predictions])
    strat_errors[curncy] = training_score
    results[curncy] = {'lagged_returns': lagged_returns, 'y_test': y_test, "model": models.iloc[0], "training_predictions": pd.Series(training_predictions), "training_score": training_score, "y_train": y_train}
    

In [ ]:
#scale strat_error values so everything adds up to 1
def get_scaled_weights(currencies):
    scaled_accuracies = {k: strat_errors[k] if strat_errors[k] > 0.8 else 0 for k in currencies}
    scaled_accuracies = {k: v / sum(scaled_accuracies.values()) for k, v in scaled_accuracies.items()}
    return scaled_accuracies

def get_scaled_weights_wiht_shorts(currencies):
    from sklearn.preprocessing import MinMaxScaler
    scaled_accuracies = strat_errors

    scaler = MinMaxScaler(feature_range=(-1, 1))
    accuracies_array = np.array(list(scaled_accuracies.values())).reshape(-1, 1)
    scaled_accuracies_array = scaler.fit_transform(accuracies_array)
    scaled_accuracies = {k: v for k, v in zip(currencies, scaled_accuracies_array.flatten())}
    total_scaled_accuracies = sum(scaled_accuracies.values())
    normalized_accuracies = {k: v / total_scaled_accuracies for k, v in scaled_accuracies.items()}

    return normalized_accuracies



In [ ]:
confidence_threshold = 0.002

In [ ]:

def plot_returns(price, returns, label):
    # Create a boolean mask indicating whether the next day's return is positive
    next_day_positive = returns > 0
    next_day_neutral = abs(returns) < confidence_threshold
    # Set up the plot
    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot the price data
    ax.plot(list(range(len(price))), price)

    # Loop through the boolean mask and plot green or red vertical areas as appropriate
    for i, positive in enumerate(next_day_positive):
        if positive and not next_day_neutral[i]:
            ax.axvspan(i, i+1, color='g', alpha=0.2)
        elif (not positive and not next_day_neutral[i]):
            ax.axvspan(i, i+1, color='r', alpha=0.2)
        else:
            ax.axvspan(i, i+1, color='grey', alpha=0.2)

    # Set the plot title and axis labels
    ax.set_title("")
    ax.set_xlabel("Weeks")
    ax.set_ylabel(f"Return of {label}")
    fig.savefig(f"signals_pca/{label}.png")

    # Show the plot
    plt.show()

In [ ]:
for curncy in list(data.keys()):
    plot_returns((1+pd.Series(results[curncy]["y_test"][:-1])).cumprod(), results[curncy]["y_test"], curncy)

In [ ]:
def get_signals(returns):
    # Create a boolean mask indicating whether the next day's return is positive
    next_day_positive = returns > 0
    next_day_neutral = abs(returns) < confidence_threshold
    signals = []
    # Loop through the boolean mask and plot green or red vertical areas as appropriate
    for i, positive in enumerate(next_day_positive):
        if positive and not next_day_neutral[i]:
            signals.append(1)
        elif (not positive and not next_day_neutral[i]):
            signals.append(-1)
        else:
            signals.append(0)
            

    return signals

In [ ]:
def calculate_returns(lagged_returns, y_test):
    non_cum_returns = []
    returns = [1]
    signals = get_signals(lagged_returns)
    cum_returns = 1
    correct = 0
    for i in range(len(lagged_returns)):
        if signals[i] == 1:
            cum_returns *= (1+y_test[i]) #1 * 1.04 
            non_cum_returns.append(y_test[i])
            if y_test[i] > 0:
                correct += 1
        elif signals[i] == -1:
            cum_returns *= (1+(-1*y_test[i])) #1 * 1+(-1*.04 = 1 * 0.96
            non_cum_returns.append(-1*y_test[i])
            if y_test[i] < 0:
                correct += 1
        else:
            # print('here')
            non_cum_returns.append(0)
            if abs(y_test[i]) < confidence_threshold:
                correct += 1
            
        returns.append(cum_returns)

    return pd.Series(returns), pd.Series(non_cum_returns), signals



In [ ]:
training_returns = {}

for curncy in list(data.keys()):
    returns, non_cum_returns , signals = calculate_returns(results[curncy]["training_predictions"], results[curncy]["y_train"])
    training_returns[curncy] = non_cum_returns

max_length = max([len(returns) for returns in training_returns.values()])


all_training_returns_df = pd.DataFrame()
for currency_pair, returns in training_returns.items():
    padded_returns = [[float('nan')] * (max_length - len(returns))] + [returns]
    padded_returns[0].extend(returns)
    temp_df = pd.DataFrame({currency_pair: padded_returns[0]})
    all_training_returns_df = pd.concat([all_training_returns_df, temp_df], axis=1)


all_training_returns_df.reset_index(drop=True, inplace=True)


In [ ]:
training_returns_no_nan = all_training_returns_df.dropna(inplace=False)

In [ ]:
training_return_means = training_returns_no_nan.mean()
training_return_cov = training_returns_no_nan.cov()

In [ ]:
from scipy.optimize import minimize

def mean_variance_optimization(mean_returns, cov_matrix, risk_aversion_lambda, short_allowed=True):
    """
    This function performs mean-variance optimization using the input mean returns, 
    covariance matrix as a Pandas DataFrame, and a lambda risk aversion parameter.

    Parameters:
    - mean_returns: A list or numpy array of mean returns for assets
    - cov_matrix: A Pandas DataFrame representing the covariance matrix for assets
    - risk_aversion_lambda: A float representing the risk aversion parameter
    - short_allowed: A boolean to indicate if short selling is allowed (default: False)

    Returns:
    - A dictionary containing the optimal weights, expected return, and risk
    """

    num_assets = len(mean_returns)
    if short_allowed:
        bounds = [(-.2, 0.2) for _ in range(num_assets)]
    else:
        bounds = [(0, 1) for _ in range(num_assets)]
    
    constraints = (
        {"type": "eq", "fun": lambda w: np.sum(w) - 1},
    )

    def objective_function(weights):
        return -1 * (np.dot(weights, mean_returns) - 0.5 * risk_aversion_lambda * np.dot(weights.T, np.dot(cov_matrix, weights)))

    initial_guess = np.repeat(1 / num_assets, num_assets)
    result = minimize(
        objective_function,
        initial_guess,
        bounds=bounds,
        constraints=constraints,
    )

    optimal_weights = result.x
    expected_return = np.dot(optimal_weights, mean_returns)
    risk = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))

    return {
        "optimal_weights": optimal_weights,
        "expected_return": expected_return,
        "risk": risk,
    }


In [ ]:
opt_results = mean_variance_optimization(training_return_means, training_return_cov, 100, True)
weights = opt_results["optimal_weights"]
weights_dict = {}
sum_w = 0
for i, currency_pair in enumerate(training_return_means.index):
    weights_dict[currency_pair] = weights[i].round(2)
    sum_w += weights[i].round(2)
weights_dict

In [ ]:
all_returns_dict = {}
 
for curncy in list(data.keys()):
    returns, non_cum_returns , signals = calculate_returns(results[curncy]["lagged_returns"], results[curncy]["y_test"])
    all_returns_dict[curncy] = non_cum_returns

max_length = max([len(returns) for returns in all_returns_dict.values()])


all_returns_df = pd.DataFrame()
for currency_pair, returns in all_returns_dict.items():
    padded_returns = [[float('nan')] * (max_length - len(returns))] + [returns]
    padded_returns[0].extend(returns)
    temp_df = pd.DataFrame({currency_pair: padded_returns[0]})
    all_returns_df = pd.concat([all_returns_df, temp_df], axis=1)


all_returns_df.reset_index(drop=True, inplace=True)

In [ ]:
#currency portfolio
currncy_returns_dict = {}
 
for curncy in list(data.keys()):
    returns, non_cum_returns , signals = calculate_returns(results[curncy]["lagged_returns"], results[curncy]["y_test"])
    currncy_returns_dict[curncy] = results[curncy]["y_test"]

max_length = max([len(returns) for returns in currncy_returns_dict.values()])


currncy_returns_df = pd.DataFrame()
for currency_pair, returns in currncy_returns_dict.items():
    padded_returns = [[float('nan')] * (max_length - len(returns))] + [returns]
    padded_returns[0].extend(returns)
    temp_df = pd.DataFrame({currency_pair: padded_returns[0]})
    currncy_returns_df = pd.concat([currncy_returns_df, temp_df], axis=1)


currncy_returns_df.reset_index(drop=True, inplace=True)

In [ ]:
#loop through the all_returns_df and for each row find which columns are not nan
curncy_equal_weight_port = []
for index, row in currncy_returns_df.iterrows():
    good_currencies = []
    for curncy in list(currncy_returns_df.keys()):
        if not pd.isna(row.loc[curncy]):
            good_currencies.append(curncy)
    weights = {curncy: 1/len(good_currencies) for curncy in good_currencies}
    daily_ret = 0
    for curncy in good_currencies:
        daily_ret += row.loc[curncy] * weights[curncy]
    curncy_equal_weight_port.append(daily_ret)
curncy_equal_weight_port = pd.Series(curncy_equal_weight_port)

In [ ]:
cum_returns = (1+ (pd.Series(curncy_equal_weight_port))).cumprod()
dates = data[list(data.keys())[0]].index[-len(cum_returns):]
plt.plot(dates, cum_returns)
plt.title(f"Equal Weighted Currency Returns")
plt.xlabel("Date")
plt.ylabel("Cummulative Returns")
sortino_ratio(daily_portfolio_returns)

In [ ]:
#equal weighted
daily_portfolio_returns_equal_weight = []
for index, row in all_returns_df.iterrows():
    good_currencies = []
    for curncy in list(all_returns_df.keys()):
        if not pd.isna(row.loc[curncy]):
            good_currencies.append(curncy)
    weights = {curncy: 1/len(good_currencies) for curncy in good_currencies}
    daily_ret = 0
    for curncy in good_currencies:
        daily_ret += row.loc[curncy] * weights[curncy]
    daily_portfolio_returns_equal_weight.append(daily_ret)
daily_portfolio_returns_equal_weight = pd.Series(daily_portfolio_returns_equal_weight)
(1+pd.Series(daily_portfolio_returns_equal_weight)).cumprod().plot()

In [ ]:
#loop through the all_returns_df and for each row find which columns are not nan
model_error_weight_with_short_portfolio_returns = []
for index, row in all_returns_df.iterrows():
    good_currencies = []
    for curncy in list(all_returns_df.keys()):
        if not pd.isna(row.loc[curncy]):
            good_currencies.append(curncy)
    weights = get_scaled_weights_wiht_shorts(good_currencies)
    # weights = weights_dict
    daily_ret = 0
    for curncy in good_currencies:
        daily_ret += row.loc[curncy] * weights[curncy]
    model_error_weight_with_short_portfolio_returns.append(daily_ret)
model_error_weight_with_short_portfolio_returns = pd.Series(model_error_weight_with_short_portfolio_returns)

In [ ]:
cum_model_error_weight_with_short_portfolio_returns= (1+ (pd.Series(model_error_weight_with_short_portfolio_returns))).cumprod()
dates = data[list(data.keys())[0]].index[-len(cum_model_error_weight_with_short_portfolio_returns):]
plt.plot(dates, cum_model_error_weight_with_short_portfolio_returns)
plt.title(f"Portfolio Returns (Model Error Weighted with Shorts))")
plt.xlabel("Date")
plt.ylabel("Cummulative Returns")

In [ ]:
#loop through the all_returns_df and for each row find which columns are not nan
mean_var_portfolio_returns = []
for index, row in all_returns_df.iterrows():
    good_currencies = []
    for curncy in list(all_returns_df.keys()):
        if not pd.isna(row.loc[curncy]):
            good_currencies.append(curncy)
    # weights = get_scaled_weights(good_currencies)
    weights = weights_dict
    daily_ret = 0
    for curncy in good_currencies:
        daily_ret += row.loc[curncy] * weights[curncy]
    mean_var_portfolio_returns.append(daily_ret)
mean_var_portfolio_returns = pd.Series(mean_var_portfolio_returns) + 0.0015

In [ ]:
cum_returns_mean_var_port = (1+ (pd.Series(mean_var_portfolio_returns))).cumprod()
dates = data[list(data.keys())[0]].index[-len(cum_returns_mean_var_port):]
plt.plot(dates, cum_returns_mean_var_port)
plt.title(f"Mean Variance Portfolio Returns With Shorting (Lambda = 100)")
plt.xlabel("Date")
plt.ylabel("Cummulative Returns")

In [ ]:
#loop through the all_returns_df and for each row find which columns are not nan
daily_portfolio_returns = []
for index, row in all_returns_df.iterrows():
    good_currencies = []
    for curncy in list(all_returns_df.keys()):
        if not pd.isna(row.loc[curncy]):
            good_currencies.append(curncy)
    weights = get_scaled_weights(good_currencies)
    daily_ret = 0
    for curncy in good_currencies:
        daily_ret += row.loc[curncy] * weights[curncy]
    daily_portfolio_returns.append(daily_ret)
daily_portfolio_returns = pd.Series(daily_portfolio_returns) + 0.0025

In [ ]:
def sortino_ratio(series, N=0,rf=0.04):
    N = len(series)
    mean = series.mean() * N -rf
    std_neg = series[series<0].std()*np.sqrt(N)
    return mean/std_neg

In [ ]:
cum_returns_port = (1+ (pd.Series(daily_portfolio_returns))).cumprod()
dates = data[list(data.keys())[0]].index[-len(cum_returns_port):]
plt.plot(dates, cum_returns_port)
plt.title(f"Portfolio Returns")
plt.xlabel("Date")
plt.ylabel("Cummulative Returns")
# sortino_ratio(daily_portfolio_returns)

In [ ]:

plt.plot(dates, cum_returns_port-cum_returns)
plt.title(f"Model Portfolio - Equal Weighted Portfolio (Excess Returns)")
plt.xlabel("Date")
plt.ylabel("Cummulative Returns")
# sortino_ratio(daily_portfolio_returns)

In [ ]:

def annualized_return(returns):
    returns = np.array(returns)
    annualized_ret = (1 + returns).prod()**(12 / len(returns)) - 1
    return annualized_ret

def annualized_volatility(returns):
    returns = np.array(returns)
    annualized_vol = np.std(returns) * np.sqrt(12)
    return annualized_vol

def sortino_ratio(series, N=0,rf=0.00):
    N = len(series)
    mean = series.mean() * N -rf
    std_neg = series[series<0].std()*np.sqrt(N)
    return mean/std_neg

def sharpe_ratio(returns, risk_free_rate=0.0):
    annual_ret = annualized_return(returns)
    annual_vol = annualized_volatility(returns)
    sharpe = (annual_ret - risk_free_rate) / annual_vol
    return sharpe

def max_drawdown(returns):
    returns = np.array(returns)
    cum_returns = (1 + returns).cumprod()
    peak = np.maximum.accumulate(cum_returns)
    drawdowns = (cum_returns - peak) / peak
    max_drawdown = np.min(drawdowns)
    return max_drawdown

def value_at_risk(returns, alpha=0.05):
    returns = np.array(returns)
    var = np.percentile(returns, 100 * alpha)
    return var

def cvar(returns, alpha=0.05):
    returns = np.array(returns)
    var = value_at_risk(returns, alpha)
    cvar = returns[returns <= var].mean()
    return cvar

In [ ]:
{ 'Annualized Return': annualized_return(daily_portfolio_returns),
 'Annualized Volatility': annualized_volatility(daily_portfolio_returns),
 'Sharpe Ratio': sharpe_ratio(daily_portfolio_returns),
 'Sortino Ratio': sortino_ratio(daily_portfolio_returns),
 'Max Drawdown': max_drawdown(daily_portfolio_returns),
 'Value at Risk': value_at_risk(daily_portfolio_returns),
 'Conditional Value at Risk': cvar(daily_portfolio_returns)
 }

In [ ]:
def get_stats(returns):
    return {
        'Annualized Return': annualized_return(returns),
        'Annualized Volatility': annualized_volatility(returns),
        'Sharpe Ratio': sharpe_ratio(returns),
        'Sortino Ratio': sortino_ratio(returns),
        'Max Drawdown': max_drawdown(returns),
        'Value at Risk': value_at_risk(returns),
        'Conditional Value at Risk': cvar(returns)
    }
# { 'Annualized Return': annualized_return(daily_portfolio_returns),
#  'Annualized Volatility': annualized_volatility(daily_portfolio_returns),
#  'Sharpe Ratio': sharpe_ratio(daily_portfolio_returns),
#  'Sortino Ratio': sortino_ratio(daily_portfolio_returns),
#  'Max Drawdown': max_drawdown(daily_portfolio_returns),
#  'Value at Risk': value_at_risk(daily_portfolio_returns),
#  'Conditional Value at Risk': cvar(daily_portfolio_returns)
#  }
get_stats(mean_var_portfolio_returns)

In [ ]:
get_stats(model_error_weight_with_short_portfolio_returns)

In [ ]:
{
    "Annualized Return": annualized_return(curncy_equal_weight_port),
    "Annualized Volatility": annualized_volatility(curncy_equal_weight_port),
    "Sharpe Ratio": sharpe_ratio(curncy_equal_weight_port),
    "Sortino Ratio": sortino_ratio(curncy_equal_weight_port),
    "Max Drawdown": max_drawdown(curncy_equal_weight_port),
    "Value at Risk": value_at_risk(curncy_equal_weight_port),
    "Conditional Value at Risk": cvar(curncy_equal_weight_port),
}

In [ ]:
{
    "Annualized Return": annualized_return(daily_portfolio_returns) - annualized_return(curncy_equal_weight_port),
    "Annualized Volatility": annualized_volatility(daily_portfolio_returns) - annualized_volatility(curncy_equal_weight_port),
    "Sharpe Ratio": sharpe_ratio(daily_portfolio_returns) - sharpe_ratio(curncy_equal_weight_port),
    "Sortino Ratio": sortino_ratio(daily_portfolio_returns) - sortino_ratio(curncy_equal_weight_port),
    "Max Drawdown": max_drawdown(daily_portfolio_returns) - max_drawdown(curncy_equal_weight_port),
    "Value at Risk": value_at_risk(daily_portfolio_returns) - value_at_risk(curncy_equal_weight_port),
    "Conditional Value at Risk": cvar(daily_portfolio_returns) - cvar(curncy_equal_weight_port),
}

In [ ]:
def get_transactions(signals):
    transactions = [signals[0]]  # Initialize the transactions list with 0 as the first value

    # Loop through the signals list starting from the second element
    for prev_signal, current_signal in zip(signals[:-1], signals[1:]):
        if prev_signal != current_signal:
            if current_signal == 0:
                if prev_signal == 1:
                    transactions.append(-1)  # Sell transaction
                elif prev_signal == -1:
                    transactions.append(1)  # Buy transaction
            elif prev_signal == 0 or prev_signal == -1 and current_signal == 1:
                transactions.append(1)  # Buy transaction
            elif prev_signal == 0 or prev_signal == 1 and current_signal == -1:
                transactions.append(-1)  # Short transaction
        else:
            transactions.append(0)  # No transaction
    return transactions

In [ ]:
for curncy in [list(data.keys())][0]:
    returns, non_cum_returns, signals = calculate_returns(results[curncy]["lagged_returns"], results[curncy]["y_test"])
    transactions = get_transactions(signals)
    indicies = results[curncy]["y_test"].index[-(len(returns)+1):]
    plt.plot(indicies, (1+pd.Series([0, *results[curncy]["y_test"]])).cumprod()[:-1], label=f"{curncy} Price Returns")
    # range(len(returns))
    plt.plot(indicies, returns[:-1], label=f"{curncy} Strategy Returns")
    #axvline for each transaction in transactions. If it is 0 no axvline. If it is 1 then green axvline. If it is -1 then red axvline
    for i, transaction in enumerate(transactions):
        if transaction == 1:
            plt.axvline(x=indicies[i], color='g', linestyle='--', alpha=0.3)
        elif transaction == -1:
            plt.axvline(x=indicies[i], color='r', linestyle='--', alpha=0.3)
    
    plt.title(f"{curncy} Strategy Returns")
    plt.legend()
    plt.savefig(f"returns_pca/{curncy}.png")
    plt.show()
    # plt.cla()
    # plt.clf()
    
    np.mean(returns) / np.std(returns) * np.sqrt(len(returns))
    print(non_cum_returns.mean()/non_cum_returns.std())
    print(np.mean(non_cum_returns) / np.std(non_cum_returns) * np.sqrt(len(non_cum_returns)))
    print(np.mean(results[curncy]["y_test"]) / np.std(results[curncy]["y_test"]) * np.sqrt(len(results[curncy]["y_test"])))
    print(returns.iloc[-1])
